In [ ]:
library(dplyr)
library(data.table)
library(reshape2)

In [ ]:
traits = c("BMI", "SBP", "DBP", "LDL", "HBA1C", "VITD", "ALT", "ALB" ,"ALP" ,"APOA", "APOB", "AST", "CAL", "CHO" ,"CREA", "CRP", "GGT", "GLUC", "HDL", "IGF1", "PHOS", "SHBG", "TBIL", "TPROT", "TRIG", "URA", "URT")

In [ ]:
fuma.path = '{location of indepedent SNPs from fuma}'
sst.path = '{location of summary statistics}'

In [ ]:
output = '{output path}'
kb = 5000000
loci.all = data.frame()
for (trait in traits){
    df.ind = fread(paste(fuma.path,trait,'/IndSigSNPs.txt', sep = ''))
    df.sst = fread(paste(sst.path,trait,'.regenie',sep=''))
    
    for (i in 1:nrow(df.ind)){
        pos.locus = df.ind$pos[i]
        chr.locus = df.ind$chr[i]
        
        if (pos.locus - kb < 0){
            start = pos.locus
        } else {
            start = pos.locus - kb
        }
        end = pos.locus + kb
        
        sst.locus = df.sst %>% filter(CHR == chr.locus) %>% filter(BP > pos.locus - kb & BP < pos.locus + kb)
        sst.locus = sst.locus %>% select(CHR,BP,SNP,ALLELE1,ALLELE0,Z)
        colnames(sst.locus) = c('CHR','BP','SNP','A1','A2','Z')
        write.table(sst.locus, paste(output,trait,'/','locus_',i,'.sst', sep = ''), col.names = TRUE, row.names = FALSE, quote = FALSE)
        
        
        locus.number = i
        locus.trait = data.frame(trait, chr.locus, pos.locus, start, end, locus.number)
        loci.all = rbind(loci.all, locus.trait)
        write.table(loci.all, paste(output,'ind_loci.pos', sep = ''), col.names = TRUE, row.names = FALSE, quote = FALSE)
    }
    
}